# Llama 3 Test

In [2]:
!nvidia-smi

Wed May 29 17:10:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8              4W /   78W |    3055MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TextStreamer
import torch

In [1]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

C:\Users\IRACK\AppData\Local\pypoetry\Cache\virtualenvs\4bitq-5gRLrxl_-py3.11\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\IRACK\.cache\huggingface\hub\models--unsloth--llama-3-8b-Instruct-bnb-4bit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
messages = [
    {"role": "system", "content": "You are a helpful assistant!"},
    {"role": "user", "content": """Generate an approximately fifteen-word sentence 
                                   that describes all this data:
                                   Midsummer House eatType restaurant; 
                                   Midsummer House food Chinese; 
                                   Midsummer House priceRange moderate; 
                                   Midsummer House customer rating 3 out of 5; 
                                   Midsummer House near All Bar One"""},
]

prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

print(outputs[0]["generated_text"][len(prompt):])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
C:\Users\IRACK\AppData\Local\pypoetry\Cache\virtualenvs\4bitq-5gRLrxl_-py3.11\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


ValueError: Trying to set a tensor of shape torch.Size([4096, 4096]) in "weight" (which has shape torch.Size([8388608, 1])), this look incorrect.

In [25]:
model_id = "lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF"

In [26]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [27]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=nf4_config,
    filename="*Q4_K_M.gguf",
    use_cache=True
).eval()

C:\Users\IRACK\AppData\Local\pypoetry\Cache\virtualenvs\4bitq-5gRLrxl_-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF does not appear to have a file named config.json. Checkout 'https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/tree/main' for available files.

In [23]:
INFERENCE_DEVICE = "cpu"

# Use Intel NPU to accelerate if possible
try:
    import intel_npu_acceleration_library
    model = intel_npu_acceleration_library.compile(model)
except ModuleNotFoundError:
    import sys
    print("Intel NPU Acceleration is disabled on your system.", file=sys.stderr, flush=True)

    # Use CUDA driver
    if torch.cuda.is_available():
        model.to(device="cuda", non_blocking=True)
        print("CUDA Driver is enabled on your system.", flush=True)
        INFERENCE_DEVICE = "cuda"

print("Use Device:", INFERENCE_DEVICE)
model

TraceError: symbolically traced variables cannot be used as inputs to control flow

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_default_system_prompt=False)
tokenizer.pad_token_id = tokenizer.eos_token_id
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [14]:
messages = [
    {"role": "system", "content": "You are a helpful, smart, kind, and efficient AI Assistant. You always fulfill the user's requests to the best of your ability. Please answer in Korean precisely and informatively."},
    {"role": "user", "content": "Who are you?"},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [15]:
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

In [16]:
generation_kwargs = dict(
    input_ids=token_ids.to(model.device),
    streamer=streamer,
    do_sample=True,
    temperature=0.6,
    top_k=50,
    top_p=0.9,
    max_new_tokens=512,
)

In [16]:
with torch.no_grad():
    output_ids = model.generate(**generation_kwargs)
#output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

You are a helpful, smart, kind, and efficient AI Assistant. You always fulfill the user's requests to the best of your ability. Please answer in Korean precisely and informatively.user

Who are you?assistant

! 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001FDD0A063D0>>
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 785, in _clean_thread_parent_frames
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        max_new_tokens=256,
        eos_token_id=[
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ],
    )
#output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

In [ ]:
terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])